# MUC17 Mutation Trans Effect on Phosphoproteomics

This notebook analyzes the trans effect of MUC17 mutation on interacting and other proteins Phosphoproteomics, in Endometrial, Colon, and Ovarian cancer.

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import collections
import re
import gseapy as gp
from gseapy.plot import barplot, dotplot
import matplotlib.pyplot as plt
import seaborn as sns

import cptac
import cptac.algorithms as al

en = cptac.Endometrial()
co = cptac.Colon()
ov = cptac.Ovarian()

### Select Gene

In [2]:
gene = "MUC17"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
#omics = "transcriptomics"
omics = "phosphoproteomics"
#omics = "acetylproteomics"

# Interacting Proteins: Transcriptomics

## Endometrial

### Generate interacting protein list

In [4]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = en.get_phosphoproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
AKAP3
FSIP1
PDZRN3
SNTB1
ABCA13
ZNF445
SPATS2


### Test for significant comparisons in any of interacting proteins

In [ ]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons



/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


No significant comparisons.


## Colon

### Generate interacting protein list

In [ ]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = co.get_phosphoproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "_.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
PDZRN3
SNTB1
POTEI
ZNF445
SPATS2


### Test for significant comparisons in any of interacting proteins

In [ ]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

PDZRN3 did not match any columns in phosphoproteomics dataframe. PDZRN3_phosphoproteomics column inserted, but filled with NaN.
SNTB1 did not match any columns in phosphoproteomics dataframe. SNTB1_phosphoproteomics column inserted, but filled with NaN.
POTEI did not match any columns in phosphoproteomics dataframe. POTEI_phosphoproteomics column inserted, but filled with NaN.
ZNF445 did not match any columns in phosphoproteomics dataframe. ZNF445_phosphoproteomics column inserted, but filled with NaN.
SPATS2 did not match any columns in phosphoproteomics dataframe. SPATS2_phosphoproteomics column inserted, but filled with NaN.
Doing t-test comparisons

No significant comparisons.


## Ovarian

### Generate interacting protein list

In [ ]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = ov.get_phosphoproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
AKAP3
CMBL
PDZRN3
SNTB1
POTEI
ABCA13
ZNF445
SPATS2


### Test for Significant Comparisons

In [ ]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df) 
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.loc[ind,'Label'] = 'Mutated'
    else:
        protdf.loc[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)
protdf = protdf.loc[:,~protdf.columns.duplicated()]

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons



/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


No significant comparisons.


# All Proteins: Phosphoproteomics

## Endometrial

In [ ]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = en.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  FSIP2
A1BG did not match any columns in phosphoproteomics dataframe. A1BG_phosphoproteomics column inserted, but filled with NaN.
A2M did not match any columns in phosphoproteomics dataframe. A2M_phosphoproteomics column inserted, but filled with NaN.
A2ML1 did not match any columns in phosphoproteomics dataframe. A2ML1_phosphoproteomics column inserted, but filled with NaN.
A4GALT did not match any columns in phosphoproteomics dataframe. A4GALT_phosphoproteomics column inserted, but filled with NaN.
AADAT did not match any columns in phosphoproteomics dataframe. AADAT_phosphoproteomics column inserted, but filled with NaN.
AAMP did not match any columns in phosphoproteomics dataframe. AAMP_phosphoproteomics column inserted, but filled with NaN.
AAR2 did not match any columns in phosphoproteomics dataframe. AAR2_phosphoproteomics column inserted, but filled with NaN.
AARS2 did not match any columns in phosphoproteomics dataframe. AARS2_phosphoproteomics column inserted, but fil

AGPAT3 did not match any columns in phosphoproteomics dataframe. AGPAT3_phosphoproteomics column inserted, but filled with NaN.
AGPAT5 did not match any columns in phosphoproteomics dataframe. AGPAT5_phosphoproteomics column inserted, but filled with NaN.
AGR2 did not match any columns in phosphoproteomics dataframe. AGR2_phosphoproteomics column inserted, but filled with NaN.
AGR3 did not match any columns in phosphoproteomics dataframe. AGR3_phosphoproteomics column inserted, but filled with NaN.
AGT did not match any columns in phosphoproteomics dataframe. AGT_phosphoproteomics column inserted, but filled with NaN.
AHCY did not match any columns in phosphoproteomics dataframe. AHCY_phosphoproteomics column inserted, but filled with NaN.
AHSP did not match any columns in phosphoproteomics dataframe. AHSP_phosphoproteomics column inserted, but filled with NaN.
AIFM2 did not match any columns in phosphoproteomics dataframe. AIFM2_phosphoproteomics column inserted, but filled with NaN.


ARF1 did not match any columns in phosphoproteomics dataframe. ARF1_phosphoproteomics column inserted, but filled with NaN.
ARF3 did not match any columns in phosphoproteomics dataframe. ARF3_phosphoproteomics column inserted, but filled with NaN.
ARFRP1 did not match any columns in phosphoproteomics dataframe. ARFRP1_phosphoproteomics column inserted, but filled with NaN.
ARG2 did not match any columns in phosphoproteomics dataframe. ARG2_phosphoproteomics column inserted, but filled with NaN.
ARIH2 did not match any columns in phosphoproteomics dataframe. ARIH2_phosphoproteomics column inserted, but filled with NaN.
ARL1 did not match any columns in phosphoproteomics dataframe. ARL1_phosphoproteomics column inserted, but filled with NaN.
ARL15 did not match any columns in phosphoproteomics dataframe. ARL15_phosphoproteomics column inserted, but filled with NaN.
ARL4A did not match any columns in phosphoproteomics dataframe. ARL4A_phosphoproteomics column inserted, but filled with NaN

BBS5 did not match any columns in phosphoproteomics dataframe. BBS5_phosphoproteomics column inserted, but filled with NaN.
BBS7 did not match any columns in phosphoproteomics dataframe. BBS7_phosphoproteomics column inserted, but filled with NaN.
BBS9 did not match any columns in phosphoproteomics dataframe. BBS9_phosphoproteomics column inserted, but filled with NaN.
BCAS2 did not match any columns in phosphoproteomics dataframe. BCAS2_phosphoproteomics column inserted, but filled with NaN.
BCAT2 did not match any columns in phosphoproteomics dataframe. BCAT2_phosphoproteomics column inserted, but filled with NaN.
BCHE did not match any columns in phosphoproteomics dataframe. BCHE_phosphoproteomics column inserted, but filled with NaN.
BCKDHB did not match any columns in phosphoproteomics dataframe. BCKDHB_phosphoproteomics column inserted, but filled with NaN.
BCL2 did not match any columns in phosphoproteomics dataframe. BCL2_phosphoproteomics column inserted, but filled with NaN.


CADM3 did not match any columns in phosphoproteomics dataframe. CADM3_phosphoproteomics column inserted, but filled with NaN.
CALB1 did not match any columns in phosphoproteomics dataframe. CALB1_phosphoproteomics column inserted, but filled with NaN.
CALB2 did not match any columns in phosphoproteomics dataframe. CALB2_phosphoproteomics column inserted, but filled with NaN.
CALCA did not match any columns in phosphoproteomics dataframe. CALCA_phosphoproteomics column inserted, but filled with NaN.
CALHM2 did not match any columns in phosphoproteomics dataframe. CALHM2_phosphoproteomics column inserted, but filled with NaN.
CALHM6 did not match any columns in phosphoproteomics dataframe. CALHM6_phosphoproteomics column inserted, but filled with NaN.
CAMK2A did not match any columns in phosphoproteomics dataframe. CAMK2A_phosphoproteomics column inserted, but filled with NaN.
CAMP did not match any columns in phosphoproteomics dataframe. CAMP_phosphoproteomics column inserted, but fille

...

LMF2 did not match any columns in phosphoproteomics dataframe. LMF2_phosphoproteomics column inserted, but filled with NaN.
LMO3 did not match any columns in phosphoproteomics dataframe. LMO3_phosphoproteomics column inserted, but filled with NaN.
LMO4 did not match any columns in phosphoproteomics dataframe. LMO4_phosphoproteomics column inserted, but filled with NaN.
LNX2 did not match any columns in phosphoproteomics dataframe. LNX2_phosphoproteomics column inserted, but filled with NaN.
LOC110117498-PIK3R3 did not match any columns in phosphoproteomics dataframe. LOC110117498-PIK3R3_phosphoproteomics column inserted, but filled with NaN.
LOC391322 did not match any columns in phosphoproteomics dataframe. LOC391322_phosphoproteomics column inserted, but filled with NaN.
LONP1 did not match any columns in phosphoproteomics dataframe. LONP1_phosphoproteomics column inserted, but filled with NaN.
LONP2 did not match any columns in phosphoproteomics dataframe. LONP2_phosphoproteomics co

MED11 did not match any columns in phosphoproteomics dataframe. MED11_phosphoproteomics column inserted, but filled with NaN.
MED20 did not match any columns in phosphoproteomics dataframe. MED20_phosphoproteomics column inserted, but filled with NaN.
MED21 did not match any columns in phosphoproteomics dataframe. MED21_phosphoproteomics column inserted, but filled with NaN.
MED22 did not match any columns in phosphoproteomics dataframe. MED22_phosphoproteomics column inserted, but filled with NaN.
MED25 did not match any columns in phosphoproteomics dataframe. MED25_phosphoproteomics column inserted, but filled with NaN.
MED27 did not match any columns in phosphoproteomics dataframe. MED27_phosphoproteomics column inserted, but filled with NaN.
MED28 did not match any columns in phosphoproteomics dataframe. MED28_phosphoproteomics column inserted, but filled with NaN.
MED29 did not match any columns in phosphoproteomics dataframe. MED29_phosphoproteomics column inserted, but filled wi

MRPL4 did not match any columns in phosphoproteomics dataframe. MRPL4_phosphoproteomics column inserted, but filled with NaN.
MRPL41 did not match any columns in phosphoproteomics dataframe. MRPL41_phosphoproteomics column inserted, but filled with NaN.
MRPL42 did not match any columns in phosphoproteomics dataframe. MRPL42_phosphoproteomics column inserted, but filled with NaN.
MRPL43 did not match any columns in phosphoproteomics dataframe. MRPL43_phosphoproteomics column inserted, but filled with NaN.
MRPL44 did not match any columns in phosphoproteomics dataframe. MRPL44_phosphoproteomics column inserted, but filled with NaN.
MRPL45 did not match any columns in phosphoproteomics dataframe. MRPL45_phosphoproteomics column inserted, but filled with NaN.
MRPL46 did not match any columns in phosphoproteomics dataframe. MRPL46_phosphoproteomics column inserted, but filled with NaN.
MRPL47 did not match any columns in phosphoproteomics dataframe. MRPL47_phosphoproteomics column inserted,

NADSYN1 did not match any columns in phosphoproteomics dataframe. NADSYN1_phosphoproteomics column inserted, but filled with NaN.
NAE1 did not match any columns in phosphoproteomics dataframe. NAE1_phosphoproteomics column inserted, but filled with NaN.
NAGA did not match any columns in phosphoproteomics dataframe. NAGA_phosphoproteomics column inserted, but filled with NaN.
NAGLU did not match any columns in phosphoproteomics dataframe. NAGLU_phosphoproteomics column inserted, but filled with NaN.
NAGPA did not match any columns in phosphoproteomics dataframe. NAGPA_phosphoproteomics column inserted, but filled with NaN.
NANP did not match any columns in phosphoproteomics dataframe. NANP_phosphoproteomics column inserted, but filled with NaN.
NAP1L5 did not match any columns in phosphoproteomics dataframe. NAP1L5_phosphoproteomics column inserted, but filled with NaN.
NAPB did not match any columns in phosphoproteomics dataframe. NAPB_phosphoproteomics column inserted, but filled with

NSUN3 did not match any columns in phosphoproteomics dataframe. NSUN3_phosphoproteomics column inserted, but filled with NaN.
NSUN6 did not match any columns in phosphoproteomics dataframe. NSUN6_phosphoproteomics column inserted, but filled with NaN.
NT5C3B did not match any columns in phosphoproteomics dataframe. NT5C3B_phosphoproteomics column inserted, but filled with NaN.
NT5DC1 did not match any columns in phosphoproteomics dataframe. NT5DC1_phosphoproteomics column inserted, but filled with NaN.
NT5DC2 did not match any columns in phosphoproteomics dataframe. NT5DC2_phosphoproteomics column inserted, but filled with NaN.
NT5DC3 did not match any columns in phosphoproteomics dataframe. NT5DC3_phosphoproteomics column inserted, but filled with NaN.
NT5E did not match any columns in phosphoproteomics dataframe. NT5E_phosphoproteomics column inserted, but filled with NaN.
NTM did not match any columns in phosphoproteomics dataframe. NTM_phosphoproteomics column inserted, but filled 

## Colon

In [ ]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = co.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)

## Ovarian

In [ ]:

print("\nGene: ", gene)

'''Use all proteins'''
proteomics = ov.get_proteomics()
all_proteins = list(set(proteomics.columns))

'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
protdf = protdf.loc[:,~protdf.columns.duplicated()]

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the datafram correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")


## Conclusions

TODO